In [1]:
import cv2
import uuid


def color_match(image1, image2):
    # Load the images
    img1 = cv2.imread(image1)
    img2 = cv2.imread(image2)

    # Resize the images to the same dimensions
    img1 = cv2.resize(img1, (img2.shape[1], img2.shape[0]))

    # Convert images to LAB color space
    img1_lab = cv2.cvtColor(img1, cv2.COLOR_BGR2LAB)
    img2_lab = cv2.cvtColor(img2, cv2.COLOR_BGR2LAB)

    # Calculate mean and standard deviation for each channel in LAB color space
    img1_mean, img1_std = cv2.meanStdDev(img1_lab)
    img2_mean, img2_std = cv2.meanStdDev(img2_lab)

    # Perform color matching by adjusting mean and standard deviation
    for i in range(3):  # Iterate over L, A, B channels
        img1_lab[:,:,i] = img1_lab[:,:,i] - img1_mean[i]
        img1_lab[:,:,i] = img1_lab[:,:,i] * (img2_std[i] / img1_std[i])
        img1_lab[:,:,i] = img1_lab[:,:,i] + img2_mean[i]

    # Convert back to BGR color space
    matched_img = cv2.cvtColor(img1_lab, cv2.COLOR_LAB2BGR)

    return matched_img

# Example usage
#image1- targeImage
#image2 - palletImage
target_image_path ='targetimage.jpeg'
pallet_image_path = 'palletimage.jpeg'
matched_image = color_match(target_image_path, pallet_image_path)

# Display the matched image
# cv2.imshow('Color Matched Image', matched_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
output_path = str(uuid.uuid4())+'_matched_image.jpg'
cv2.imwrite(output_path, matched_image)
print(f"Color matched image saved to {output_path}")


Color matched image saved to 2daab383-be0b-4786-bdb3-8b1e39234a27_matched_image.jpg


In [3]:
import cv2
import numpy as np

# Define functions
def core_processing(targetf, sourcef, CrossCovarianceLimit, ReshapingIterations, ShaderVal):
    Lab = cv2.cvtColor(targetf, cv2.COLOR_BGR2Lab)
    sLab = cv2.cvtColor(sourcef, cv2.COLOR_BGR2Lab)

    tmean, tdev = cv2.meanStdDev(Lab)
    smean, sdev = cv2.meanStdDev(sLab)

    Lab = [(Lab[i] - tmean[i]) / tdev[i] for i in range(3)]
    sLab = [(sLab[i] - smean[i]) / sdev[i] for i in range(3)]

    jcount = ReshapingIterations
    while jcount > int((ReshapingIterations + 1) / 2):
        Lab[1] = channel_condition(Lab[1], sLab[1])
        Lab[2] = channel_condition(Lab[2], sLab[2])
        jcount -= 1

    if CrossCovarianceLimit != 0.0:
        targetf = adjust_covariance(Lab, sLab, CrossCovarianceLimit)
        Lab = cv2.split(targetf)

    while jcount > 0:
        Lab[1] = channel_condition(Lab[1], sLab[1])
        Lab[2] = channel_condition(Lab[2], sLab[2])
        jcount -= 1

    Lab[1] = Lab[1] * sdev[1] + smean[1]
    Lab[2] = Lab[2] * sdev[2] + smean[2]

    Lab[0] = Lab[0] * (ShaderVal * sdev[0] + (1.0 - ShaderVal) * tdev[0]) + ShaderVal * smean[0] + (
            1.0 - ShaderVal) * tmean[0]

    resultant = cv2.merge(Lab)
    resultant = cv2.cvtColor(resultant, cv2.COLOR_Lab2BGR)
    return resultant

def adjust_covariance(Lab, sLab, covLim):
    if covLim != 0.0:
        tcrosscorr = np.mean(Lab[1] * Lab[2])
        scrosscorr = np.mean(sLab[1] * sLab[2])

        W1 = 0.5 * np.sqrt((1 + scrosscorr) / (1 + tcrosscorr)) + 0.5 * np.sqrt((1 - scrosscorr) / (1 - tcrosscorr))
        W2 = 0.5 * np.sqrt((1 + scrosscorr) / (1 + tcrosscorr)) - 0.5 * np.sqrt((1 - scrosscorr) / (1 - tcrosscorr))

        W2 = np.sign(W2) * min(abs(W2), abs(covLim) * abs(W1))
        norm = 1.0 / np.sqrt(W1 * W1 + W2 * W2 + 2 * W1 * W2 * tcrosscorr)
        W1 = W1 * norm
        W2 = W2 * norm

        z1 = Lab[1].copy()
        Lab[1] = W1 * z1 + W2 * Lab[2]
        Lab[2] = W1 * Lab[2] + W2 * z1

    temp1 = cv2.merge(Lab)
    return temp1

def channel_condition(Chan, sChan):
    mask = cv2.threshold(sChan, 0, 255, cv2.THRESH_BINARY)[1].astype(np.uint8)

    smeanU, _ = cv2.meanStdDev(sChan, mask=mask)
    smeanL, _ = cv2.meanStdDev(sChan, mask=cv2.bitwise_not(mask))
    tmeanU, _ = cv2.meanStdDev(Chan, mask=mask)
    tmeanL, _ = cv2.meanStdDev(Chan, mask=cv2.bitwise_not(mask))

    k = np.sqrt(np.sqrt(smeanU / tmeanU))
    ChanU = (1 + (Chan - tmeanU) * (k - 1)).astype(np.float32)
    k = np.sqrt(np.sqrt(smeanL / tmeanL))
    ChanL = (1 + (Chan - tmeanL) * (k - 1)).astype(np.float32)

    Chan = np.zeros_like(Chan, dtype=np.float32)
    Chan = cv2.addWeighted(Chan, 1, ChanU, 1, 0, mask=mask)
    Chan = cv2.addWeighted(Chan, 1, ChanL, 1, 0, mask=cv2.bitwise_not(mask))

    tmean, tdev = cv2.meanStdDev(Chan)
    Chan = (Chan - tmean) / tdev
    tmean, tdev = cv2.meanStdDev(Chan)

    return Chan

# Main functio

In [4]:
CrossCovarianceLimit = 0.5
ReshapingIterations = 1
ShaderVal = 0.5

targetname = "images/Flowers_target.jpg"
sourcename = "images/Flowers_source.jpg"

target_image_path ='targetimage.jpeg'
pallet_image_path = 'palletimage.jpeg'

target = cv2.imread(target_image_path, 1)
source = cv2.imread(pallet_image_path, 1)

savedtf = target.copy()

targetf = core_processing(target, source, CrossCovarianceLimit, ReshapingIterations, ShaderVal)

result = cv2.convertScaleAbs(targetf, alpha=(255.0))

#cv2.imshow("processed image", result)
cv2.imwrite("images/processed.jpg", result)

#cv2.waitKey(0)
#cv2.destroyAllWindows()

error: OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/core/src/matrix_iterator.cpp:71: error: (-215:Assertion failed) A.size == arrays[i0]->size in function 'init'
